In [ ]:
!pip install python-dotenv
!pip install srai[all]
!pip install folium matplotlib mapclassify

In [ ]:
!pip install pgvector  # Install the pgvector module
!pip install geoalchemy2  # Install the geoalchemy2 module

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, Float, MetaData
from sqlalchemy.dialects.postgresql import ARRAY
import geopandas as gpd
from pgvector.sqlalchemy import Vector
from geoalchemy2 import Geometry
from sqlalchemy import text, insert
import numpy as np

In [ ]:
from srai.embedders import Hex2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.plotting import plot_regions, plot_numeric_data

loader = OSMPbfLoader()
regionalizer = H3Regionalizer(resolution=11)
joiner = IntersectionJoiner()

area = geocode_to_region_gdf("City of London")
features = loader.load(area, HEX2VEC_FILTER)
regions = regionalizer.transform(area)
joint = joiner.transform(regions, features)

In [ ]:
embedder = Hex2VecEmbedder()
neighbourhood = H3Neighbourhood(regions_gdf=regions)

In [ ]:
embedder = Hex2VecEmbedder([15, 10, 3])

# Option 1: fit and transform
embedder.fit(regions, features, joint, neighbourhood, batch_size=128)
embeddings = embedder.transform(regions, features, joint)

# Option 2: fit_transform
# embeddings = lightning_model.fit_transform(regions, features, joint, neighbourhood, batch_size=128)

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 1, embeddings, map=folium_map)

In [ ]:
embeddings

In [ ]:
embeddings_vector = embeddings.to_numpy()
embeddings_vector.shape

In [ ]:
embeddings_df = pd.DataFrame({
    'region': regions['geometry'],
    'embeddings': embeddings_vector.tolist()
})

In [ ]:
#convert embeddings_df to gdf

embeddings_gdf = gpd.GeoDataFrame(embeddings_df, geometry='region')
embeddings_gdf.head()

In [ ]:
records = []
for _, row in embeddings_gdf.iterrows():
    records.append({
        'region': row['region'].wkt,  # Convert geometry to WKT format for insertion
        'embeddings': np.array(row['embeddings']).tolist()  # Ensure it's a list
    })

Save the embeddings in the GeoBase

In [ ]:
from google.colab import userdata
db_string = userdata.get('db_string')

In [ ]:
# Create a connection to the database
engine = create_engine(db_string)

In [ ]:
create_table_sql = """
BEGIN;
CREATE TABLE embeddings_vector (
    id SERIAL PRIMARY KEY,
    region GEOMETRY(POLYGON, 4326),
    embeddings VECTOR(3)
);
COMMIT;
"""

In [ ]:
with engine.connect() as connection:
    connection.execute(text(create_table_sql))

In [ ]:
# Test the connection
try:
    connection = engine.connect()
    print("Connection to the database successful!")
    connection.close()
except Exception as e:
    print("Connection failed:", e)


In [ ]:
# Define the table metadata
metadata = MetaData()

# Define the table
malmo_embeddings = Table(
    'embeddings_vector', metadata,
    Column('id', Integer, primary_key=True),
    Column('region', Geometry('GEOMETRY', srid=4326)),
    Column('embeddings', Vector(3))
)

In [ ]:
metadata.create_all(engine)

In [ ]:
#save data into embeddings_vector

records = []
for _, row in embeddings_gdf.iterrows():
    records.append({
        'region': row['region'].wkt,  # Convert geometry to WKT format for insertion
        'embeddings': np.array(row['embeddings']).tolist()  # Ensure it's a list
    })
print(records[0])

In [ ]:
with engine.connect() as conn:
    with conn.begin():
      conn.execute(insert(malmo_embeddings), records)
      conn.commit()

In [ ]:
#we can save in geobase in two ways one is

# Convert 'region' column to WKT format
# embeddings_gdf['region'] = embeddings_gdf['region'].apply(lambda geom: geom.wkt)

# Use to_sql to save the data
embeddings_gdf.to_sql('embeddings_vector', engine, if_exists='append', index=False, dtype={ 'embeddings': ARRAY(Float)})

In [ ]:
query = "SELECT * FROM embeddings_vector"
read_df = gpd.read_postgis(query, engine,geom_col='region')

In [ ]:
# Check the type of each column
print(read_df.dtypes)

# Verify if the 'embeddings' column can be converted to a numeric vector
import ast
try:
    read_df['embeddings'] = read_df['embeddings'].apply(ast.literal_eval)
    print("The 'embeddings' column can be converted to a numeric vector.")
except Exception as e:
    print(f"Error converting 'embeddings' column: {e}")
